In [ ]:
!python -m pip install pyyaml==5.1
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [ ]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

In [ ]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
!unzip data.zip > /dev/null > /dev/null

In [ ]:
config_file_path = "COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"
checkpoint_url = "COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"

output_dir = "./output/object_detection"
num_classes = 1

device = "cuda"

train_dataset_name = "train_set"
train_images_path = "./data/train"
train_json_annot_path = "./data/train.json"

test_dataset_name = "test_set"
test_images_path = "./data/test"
test_json_annot_path = "./data/test.json"

cfg_save_path = "OD_mask_rcnn_101_50_cfg.pickle"

In [ ]:
from detectron2.data.datasets import register_coco_instances

In [ ]:
register_coco_instances(name = train_dataset_name, metadata = {}, json_file = train_json_annot_path, image_root = train_images_path)
register_coco_instances(name = test_dataset_name, metadata = {}, json_file = test_json_annot_path, image_root = test_images_path)

In [ ]:
def plot_samples(dataset_name, n = 1):
  dataset_custom = DatasetCatalog.get(dataset_name)
  dataset_custom_metadata = MetadataCatalog.get(dataset_name)

  for s in random.sample(dataset_custom, n):
    img = cv2.imread(s["file_name"])
    v = Visualizer(img[:,:,::-1], metadata = dataset_custom_metadata, scale = 0.5)
    v = v.draw_dataset_dict(s)
    plt.figure(figsize=(15,20))
    plt.imshow(v.get_image())
    plt.show()

In [ ]:
plot_samples(dataset_name=train_dataset_name, n=20)

In [ ]:
def get_traing_cfg(
    config_file_path,
    checkpoint_url,
    train_dataset_name,
    test_dataset_name,
    num_classes,
    device,
    output_dir
):
  cfg = get_cfg()

  cfg.merge_from_file(model_zoo.get_config_file(config_file_path))
  cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(checkpoint_url)

  cfg.DATASETS.TRAIN = (train_dataset_name,)
  cfg.DATASETS.TEST = (test_dataset_name,)

  cfg.DATALOADER.NUM_WORKERS = 2

  cfg.SOLVER.IMS_PER_BATCH = 2
  cfg.SOLVER.BASE_LR = 0.00025
  cfg.SOLVER.MAX_ITER = 500
  cfg.SOLVER.STEPS = []

  cfg.MODEL.ROI_HEADS.NUM_CLASSES = num_classes
  cfg.MODEL.DEVICE = device

  cfg.OUTPUT_DIR = output_dir

  return cfg

In [ ]:
import pickle
from detectron2.engine import DefaultTrainer

In [ ]:
def main():
  cfg = get_traing_cfg(
      config_file_path,
      checkpoint_url,
      train_dataset_name,
      test_dataset_name,
      num_classes,
      device,
      output_dir
  )

  with open(cfg_save_path, 'wb') as f:
    pickle.dump(cfg, f, protocol=pickle.HIGHEST_PROTOCOL)

  os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

  trainer = DefaultTrainer(cfg)
  trainer.resume_or_load(resume=False)

  trainer.train()

In [ ]:
main()

In [ ]:
with open(cfg_save_path, 'rb') as f:
  cfg = pickle.load(f)

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, 'model_final.pth')

cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.1

predictor = DefaultPredictor(cfg)

In [ ]:
# FUNCTION 1 - display of N sample images (original ones from TRAIN data with ANNOTATION(s))

def plot_samples(dataset_name, n = 1):
  dataset_custom = DatasetCatalog.get(dataset_name)
  dataset_custom_metadata = MetadataCatalog.get(dataset_name)

  for s in random.sample(dataset_custom, n):
    img = cv2.imread(s["file_name"])
    v = Visualizer(img[:,:,::-1], metadata = dataset_custom_metadata, scale = 0.5)
    v = v.draw_dataset_dict(s)
    plt.figure(figsize=(15,20))
    plt.imshow(v.get_image())
    plt.show()

In [ ]:
# FUNCTION 2 - display image with original annotations (i.e., original BoundingBox)
# dataset_name - train_dataset_name OR test_dataset_name

def plot_annotation(dataset_name, image_path, num=0):
  dataset_custom = DatasetCatalog.get(dataset_name)
  dataset_custom_metadata = MetadataCatalog.get(dataset_name)

  img = cv2.imread(image_path)
  v = Visualizer(img[:,:,::-1], metadata=dataset_custom_metadata, scale = 0.5)
  v = v.draw_dataset_dict(dataset_custom[num])
  plt.figure(figsize=(15, 20))
  plt.imshow(v.get_image())
  plt.show() 

In [ ]:
from detectron2.utils.visualizer import ColorMode
# FUNCTION 3 - make and show predictions on selected IMAGE
def on_image_draw(image_path, predictor):
  im = cv2.imread(image_path)
  outputs = predictor(im)
  v = Visualizer(im[:,:,::-1], metadata={}, scale = 0.5, instance_mode=ColorMode.SEGMENTATION)
  v = v.draw_instance_predictions(outputs["instances"].to("cpu"))

  plt.figure(figsize=(15,20))
  plt.imshow(v.get_image())
  plt.show()

In [ ]:
# FUNCTION 4 - get SCORE(s) and COORDINATES of BoundingBox(es)
def on_image_get_points_scores(image_path, predictor):
  im = cv2.imread(image_path)
  outputs = predictor(im)

  scores = outputs['instances'].scores
  scores_all = []

  for i in range(len(scores)):
    scores_all.append(scores[i].item())

  boxes = outputs['instances'].pred_boxes.tensor.cpu().numpy()

  return (scores_all, boxes)

In [ ]:
# FUNCTION 5 - CROP of ROI(s) out of the whole IMAGE
from PIL import Image

def crop_object(image, box):
  x_top_left = box[0]
  y_top_left = box[1]
  x_bottom_right = box[2]
  y_bottom_right = box[3]

  x_center = (x_top_left + x_bottom_right) / 2
  y_center = (y_top_left + y_bottom_right) / 2

  crop_img = image.crop((int(x_top_left), int(y_top_left), int(x_bottom_right), int(y_bottom_right)))

  return crop_img

In [ ]:
check_image_path_test1 = './data/test/20.png'

example_image_test_1 = cv2.imread(check_image_path_test1)
plt.figure(figsize=(15,20))
plt.imshow(cv2.cvtColor(example_image_test_1, cv2.COLOR_BGR2RGB))

In [ ]:
on_image_draw(check_image_path_test1, predictor)

In [ ]:
scores_1, boxes_1 = on_image_get_points_scores(check_image_path_test1, predictor)

In [ ]:
for i in range(len(scores_1)):
  print(f'BOX = ({boxes_1[i]}) \t SCORES = {scores_1[i]}')

In [ ]:
testing_example_image_converted_1 = cv2.cvtColor(example_image_test_1, cv2.COLOR_BGR2RGB)
image_pil_1 = Image.fromarray(testing_example_image_converted_1)

for i in range(len(boxes_1)):
  box = boxes_1[i]
  crop_image_1 = crop_object(image_pil_1, box)

  image_np_1 = np.asarray(crop_image_1)

  plt.figure(figsize=(12, 8))
  plt.imshow(image_np_1)